In [ ]:
# forked repo for ConvGRU, this repo includes ConvSTAR module that I modified based on ConvGRU
!git clone https://github.com/Niantangliu/ConvGRU.git
%cd ConvGRU

In [ ]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()
tf.compat.v1.experimental.output_all_intermediates(True)
from tensorflow import keras
#from keras import Sequential
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization, ConvLSTM2D,SeparableConv2D,ConvSTAR2D,ConvGRU2D,Add,Subtract,AveragePooling3D, MaxPooling3D, Average
from keras.layers import Dropout, Input ,LeakyReLU
from keras.layers import concatenate,Concatenate
from keras.activations import relu
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
#from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from time import time


from keras.utils import plot_model

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

from operator import truediv
from keras.utils import np_utils

from plotly.offline import init_notebook_mode
from keras.layers import Bidirectional
from keras import regularizers

import numpy as np
import matplotlib.pyplot as plt

# 3D-2D CNN

In [ ]:

input_layer = Input((11,11,26,3))
## convolutional layers
conv_layer1 = Conv3D(filters=32, kernel_size=(3, 3, 3),activation='relu')(input_layer)
bn1 = BatchNormalization()(conv_layer1)
conv_layer2 = Conv3D(filters=32, kernel_size=(3, 3, 3) ,activation='relu')(bn1)
bn2 = BatchNormalization()(conv_layer2)
conv_layer3 = Conv3D(filters=64, kernel_size=(3, 3, 3),activation='relu')(bn2)
bn3 = BatchNormalization()(conv_layer3)
#print(conv_layer3._keras_shape)
conv3d_shape = conv_layer3.shape
conv_layer4 = Reshape((conv3d_shape[1], conv3d_shape[2], 20*conv3d_shape[4]))(bn3)
conv_layer5 = Conv2D(filters=128, kernel_size=(3,3), activation='relu')(conv_layer4)
bn4 = BatchNormalization()(conv_layer5)

flatten_layer = Flatten()(bn4)

## fully connected layers
dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
output_layer = Dense(units=5, activation='softmax')(dense_layer2)

# 3D-CNN

In [ ]:

input_layer = Input((11,11,26,3))

conv_layer1 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu', padding='same')(input_layer)
p1 = MaxPooling3D(pool_size = (2,2,1))(conv_layer1)
conv_layer2 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu', padding='same')(p1)
p2 = MaxPooling3D(pool_size = (2,2,1))(conv_layer2)
conv_layer3 = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu', padding='same')(p2)
p3 = MaxPooling3D(pool_size = (2,2,1))(conv_layer3)

flatten_layer = Flatten()(p3)
dense_layer= Dense(units=64, activation='relu')(flatten_layer)
output_layer = Dense(units=5, activation='softmax')(dense_layer)


# TCNN

In [ ]:
input_layer = Input((26,3))
conv_layer1 = Conv1D(filters = 64, kernel_size = 5)(input_layer)
Dplayer1 = Dropout(0.5)(conv_layer1)
conv_layer2 = Conv1D(filters = 64, kernel_size = 5)(Dplayer1)
Dplayer2 = Dropout(0.5)(conv_layer2)
conv_layer3 = Conv1D(filters = 64, kernel_size = 5)(Dplayer2)
Dplayer3 = Dropout(0.5)(conv_layer3)

flatten_layer = Flatten()(Dplayer3)
dense_layer = Dense(units=512, activation='relu')(flatten_layer)
Dplayer4 = Dropout(0.5)(dense_layer)

output_layer = Dense(units=5, activation='softmax')(Dplayer4)

# ConvSTAR

In [ ]:
input_layer = Input((11,11,26,3))
x = Reshape((26,11,11,3))(input_layer)
## convolutional layers
conv_layer1 = ConvSTAR2D(filters=64, kernel_size=(3, 3),activation='relu',return_sequences=True)(x)
conv_layer2 = ConvSTAR2D(filters=64, kernel_size=(3, 3) ,activation='relu',return_sequences=True)(conv_layer1)
conv_layer3 = ConvSTAR2D(filters=64, kernel_size=(3, 3) ,activation='relu')(conv_layer2)

#conv_layer4 = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(conv_layer3)
#conv_layer5 = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(conv_layer4)

flatten_layer = Flatten()(conv_layer3)

## fully connected layers
# dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
# dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=64, activation='relu')(flatten_layer)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=5, activation='softmax')(dense_layer2)

# 3D-ConvSTAR

In [ ]:
input_layer = Input((11,11,26,3))
## convolutional layers
conv_layer1 = Conv3D(filters=32, kernel_size=(3, 3, 3),strides=(1, 1, 1),activation='relu',padding='same')(input_layer)
#Dp1 = Dropout(0.4)(conv_layer1)
conv_layer2 = Conv3D(filters=32, kernel_size=(3, 3, 3),strides=(1, 1, 1),activation='relu',padding='same')(conv_layer1)
#Dp2 = Dropout(0.4)(conv_layer2)
conv_layer3 = Conv3D(filters=64, kernel_size=(3, 3, 3),strides=(1, 1, 1),activation='relu',padding='same')(conv_layer2)
#Dp3 = Dropout(0.4)(conv_layer3)

x = Reshape((26,11,11,64))(conv_layer3)
x = Bidirectional(ConvSTAR2D(filters=64, kernel_size=(3, 3),activation='relu', return_sequences=True),merge_mode='sum')(x)
x = Bidirectional(ConvSTAR2D(filters=64, kernel_size=(3, 3),activation='relu', return_sequences=True),merge_mode='sum')(x)
x = Bidirectional(ConvSTAR2D(filters=64, kernel_size=(3, 3),activation='relu', return_sequences=True),merge_mode='sum')(x)

#Dp4 = Dropout(0.4)(x)

#x = Bidirectional(ConvSTAR2D(filters=64, kernel_size=(3, 3),activation='relu', return_sequences=True),merge_mode='sum')(x)
#x = Bidirectional(ConvSTAR2D(filters=64, kernel_size=(3, 3),activation='relu',return_sequences=True),merge_mode='sum')(x)

conv3d_shape = x.shape
conv2d = Reshape((conv3d_shape[2], conv3d_shape[3], 26*conv3d_shape[4]))(x)
conv_layer4 = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(conv2d)
#Dp5 = Dropout(0.4)(conv_layer4)


flatten_layer = Flatten()(conv_layer4)

dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=5, activation='softmax')(dense_layer2)